# Activity 7 - Decorator and Closure

## Decorator



### Function Object 

In python, functions can be treated just like a regular object. It can be passed as arguments in other functions. 

In the example below, `operation` is a callable function and is treated same as other two numerical arguments `x` and `y`. 

In [1]:
def apply_operation(operation, x, y):
    return operation(x, y)

def add(a, b):
    return a + b

def multiply(a, b):
    return a * b

result1 = apply_operation(add, 3, 5)
print(result1)  

result2 = apply_operation(multiply, 3, 5)
print(result2)  


8
15


### What is Decorator 

A decorator is a **special type of function** that allows you to modify the behavior of another function without changing its source code. Decorators provide a way to add functionality to existing functions dynamically.

Decorators are denoted by the `@` symbol followed by the decorator function name. They are applied directly above the definition of the function or class being decorated. 

A decorator is a callable that takes another function as an argument (the decorated
function).

A decorator may perform some processing with the decorated function, and returns
it or replaces it with another function or callable object.

Assume we have a function `decorate` which takes another function as argument and print its name. 

In [2]:
def decorate(func):
    print(f"decorating {func.__name__}")
    return func

Given a dummy function `target`, we can create a new function `target` by calling `decorate` passing the `target`. 

In [3]:
def target():
    print('running target()')
    
target = decorate(target)

target()

decorating target
running target()


We can write above code in another way by using syntax sugar `@`:

In [4]:
@decorate
def target():
    print('running target()')  

target()

decorating target
running target()


We now have succefully created our first decorator. It is not a new magic stuff, but a syntax sugar to rewrite the code in another way. 

When the decorated function is called or instantiated, the decorator modifies or wraps around its behavior. 

Here is anohter example replacing the decorated function. 

In [5]:
def decorate(func):

    def add_100_more(*args):
        result = func(*args) # call get_sum as usual
        result += 100
        return result # add 100 more and return the new value 

    return add_100_more # return a new function replacing get_sum 

@decorate
def get_sum(x, y):
    return x+y  

get_sum(1, 2)

103

Running the code below to verify that get_sum has been replaced by add_100_more. 

In [6]:
get_sum.__name__

'add_100_more'

**Question**

Finish the decorator function below to add additional features to `divide`, such as explanation of the division, handling zero division, or something else.

In [23]:
def decorated_divide(func):
    def inner(a,b):
      if b == 0:
        return
      else:
        return func(a,b)
    return inner
    

@decorated_divide
def divide(a, b):
    print(a/b)


divide(2,5)
#divide(2,0)

0.4


**Question**

In the code cell below, there are three functions f1 to f3 that all return strings. We want to require them return strings in uppercase but we do not want to modify their function bodies. 

Please define a decorator and apply it to three functions instead. The expected output is

```
HELLO
ABC
HELLO WESTMINSTER
```

In [30]:
# Define your decorator function here and apply them on f1-f3.

def decorated_f(func):
  def inner(*args):
    result = func(*args)
    result = result.upper()
    return result
  return inner

@decorated_f
def f1():
    return "hello"

@decorated_f
def f2(L: list):
    return ''.join(L)

@decorated_f
def f3(name):
    return "hello" + " " + name

print(f1())
print(f2(['a', 'b', 'c']))
print(f3("Westminster"))

HELLO
ABC
HELLO WESTMINSTER


### Implentation Time

A key feature of decorators is that they run right after the decorated function is
defined.

In [31]:
registry = []  # <1>

def register(func):  # <2>
    print(f'running register({func})')  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

print('registry ->', registry)

running register(<function f1 at 0x7f496018d900>)
running register(<function f2 at 0x7f496018e7a0>)
registry -> [<function f1 at 0x7f496018d900>, <function f2 at 0x7f496018e7a0>]


The decorators of f1 and f2 are implemented right after f1 and f2 are created. And the list `registry` has been filled. 

In [32]:
def main():  # <8>
    print('running main()')
    f1()
    f2()
    f3()

main()  # <9>

running main()
running f1()
running f2()
running f3()


The function body of f1 to f3 are implemented until they are called explicitly. 

### Clock Application 

Decorators are commonly used for tasks such as logging, timing, input validation, caching, and adding functionality to classes. They provide a clean and elegant way to extend or modify the behavior of existing code without directly modifying its source.

In the example below, we create a decoractor `clock` to measure the running time of a giving function `func`. 

In [33]:
import time

def clock(func):
    def clocked(*args):  # accept whatever arguments func has
        t0 = time.perf_counter() # current time
        result = func(*args)  # run func as normal 
        elapsed = time.perf_counter() - t0 # elapsed time 
        name = func.__name__ # name of func 
        arg_str = ', '.join(repr(arg) for arg in args) # patch arguments together 
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result # return the results from func as normal 
    return clocked  # return the decorated function 

A snooze function below does nothing but wait for seconds to complete. 

In [34]:
@clock
def snooze(seconds):
    time.sleep(seconds)

snooze is decorated by clock. Now whenever snooze is called, its running time is measured and printed. 

In [35]:
snooze(2)

[2.00213637s] snooze(2) -> None


A factorial function is decorated below. 

In [36]:
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

In [37]:
factorial(4)

[0.00000164s] factorial(1) -> 1
[0.00096565s] factorial(2) -> 2
[0.00141456s] factorial(3) -> 6
[0.00184326s] factorial(4) -> 24


24

**Question**

Why `factorial(4)` prints the running time four time, instead of one time?  

This is the typical behavior of a decorator: it replaces the decorated function with a
new function that accepts the same arguments and (usually) returns whatever the
decorated function was supposed to return, while also doing some extra processing. 

**Question**

Read the following code on decorator `authenticate`. What is the purpose of using this decorator?

In [38]:
def authenticate(func):
    def wrapper(user, *args, **kwargs):
        if user.is_authenticated():
            if user.has_permission(func.__name__):
                return func(user, *args, **kwargs)
            else:
                raise PermissionError("User does not have permission to access this resource.")
        else:
            raise PermissionError("User is not authenticated.")
    return wrapper

@authenticate
def protected_resource(user):
    # Function code here
    pass

current_user = get_current_user()  # Retrieve the current user

protected_resource(current_user)


NameError: ignored

### Built-In Decorator 

#### functools.wrap

In Python, `functools.wrap` is a function decorator provided by the `functools` module. It is used to create a new function that "wraps" or decorates another function, allowing you to add additional functionality or modify the behavior of the wrapped function.

The `functools.wrap` decorator takes a function, referred to as the "wrapper function," and wraps it around another function, referred to as the "original function." The wrapper function typically performs some additional tasks before or after calling the original function, such as logging, input validation, or modifying the return value.

The `functools.wrap` decorator is commonly used when creating decorators in Python to ensure that the wrapped function retains its important metadata and helps preserve the behavior of the original function even when it is decorated.

If you don't use `functools.wraps` in your code when creating a decorator, the behavior of the decorated function may be affected in terms of metadata and introspection.

When you apply a decorator without using `functools.wraps`, the decorated function will effectively be replaced by the wrapper function, losing its original name, docstring, and other attributes. This can lead to confusion and make it harder to debug and understand the code.

Consider the following example to illustrate the difference:

In [39]:
def decorator(func):
    def wrapper(*args, **kwargs):
        print("Before calling the original function")
        result = func(*args, **kwargs)
        print("After calling the original function")
        return result
    return wrapper

@decorator
def original_function(x, y):
    """Original function"""
    return x + y

print(original_function.__name__)  
print(original_function.__doc__)   

wrapper
None


The name and docstring of original_function have been replaced. But if we only want to decorate original_function and retain its name and docstring, we can simply use @functools.wraps:

In [40]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print("Before calling the original function")
        result = func(*args, **kwargs)
        print("After calling the original function")
        return result
    return wrapper

@decorator
def original_function(x, y):
    """Original function"""
    return x + y

print(original_function.__name__)  
print(original_function.__doc__)   

original_function
Original function


#### functools.cache

The functools.cache decorator implements memoization:5 an optimization technique
that works by saving the results of previous invocations of an expensive function,
avoiding repeat computations on previously used arguments.

A good demonstration is to apply @cache to the painfully slow recursive function to
generate the nth number in the Fibonacci sequence.

In [41]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

fibonacci(6)

[0.00000091s] fibonacci(0) -> 0
[0.00000086s] fibonacci(1) -> 1
[0.00108197s] fibonacci(2) -> 1
[0.00000043s] fibonacci(1) -> 1
[0.00000080s] fibonacci(0) -> 0
[0.00000037s] fibonacci(1) -> 1
[0.00003169s] fibonacci(2) -> 1
[0.00006245s] fibonacci(3) -> 2
[0.00118307s] fibonacci(4) -> 3
[0.00000033s] fibonacci(1) -> 1
[0.00000037s] fibonacci(0) -> 0
[0.00000036s] fibonacci(1) -> 1
[0.00003916s] fibonacci(2) -> 1
[0.00006908s] fibonacci(3) -> 2
[0.00000031s] fibonacci(0) -> 0
[0.00000034s] fibonacci(1) -> 1
[0.00002874s] fibonacci(2) -> 1
[0.00000038s] fibonacci(1) -> 1
[0.00001448s] fibonacci(0) -> 0
[0.00000035s] fibonacci(1) -> 1
[0.00004556s] fibonacci(2) -> 1
[0.00007652s] fibonacci(3) -> 2
[0.00013448s] fibonacci(4) -> 3
[0.00023343s] fibonacci(5) -> 5
[0.00144713s] fibonacci(6) -> 8


8

**Question**

Why are there so many repeated calculation?

We can stack functools.cache over clock to save and avoid repeated calculations. 

In [42]:
@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

fibonacci(6)

[0.00000115s] fibonacci(0) -> 0
[0.00000141s] fibonacci(1) -> 1
[0.00221023s] fibonacci(2) -> 1
[0.00000137s] fibonacci(3) -> 2
[0.00227046s] fibonacci(4) -> 3
[0.00000109s] fibonacci(5) -> 5
[0.00231275s] fibonacci(6) -> 8


8

The stacked decorators:

```
@alpha
@beta
def myfunc():
    pass
```

works in this order:

```
alpha(beta(myfunc))
```

### Decorators on class

In Python, decorators can also be applied to classes to modify their behavior or add additional functionality. When a decorator is applied to a class, it is invoked at the time of class definition and alters the class object itself. The decorator function receives the class object as its argument and can perform modifications on the class or replace it with a new class.

Here's an example to illustrate how decorators can be used with classes:


In [43]:
def add_custom_method(cls):
    def custom_method(self):
        print("Custom method called!")

    cls.custom_method = custom_method
    return cls

@add_custom_method
class MyClass:
    pass

obj = MyClass()
obj.custom_method()  

Custom method called!


By applying the `@add_custom_method` decorator above the `MyClass` class definition, the decorator is invoked with the class object as its argument. It adds the `custom_method` to the class, allowing instances of the class to call this method.

Decorators applied to classes can be used to add additional methods, modify existing methods, add attributes, or perform any other desired changes to the class object. They provide a way to extend or customize the behavior of classes without modifying their original implementation.

## Closure

### Variable Scope 

**Question**

Is there an error in the code below:

In [44]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: ignored

In the example above,  we define and test a function that reads two variables: a local variable
a—defined as function parameter—and variable b that is not defined anywhere in the
function.

We can fix it by defining b somewhere in the code.

In [45]:
b = 6
f1(3)

3
6


**Question**

Is there an error in the code below:

In [48]:
b = 6

def f2(a):
    global b
    print(a)
    print(b)
    b = 9

f2(3)

3
6


When Python compiles the body of the function, it decides that b is a
local variable because it is assigned within the function.

This is not a bug, but a design choice: Python does not require you to declare variables,
but assumes that a variable assigned in the body of a function is local.

We can fix it by declare b as global inside the function. 

In [49]:
b = 6

def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In the preceding examples, we can see two scopes in action:
- The module global scope

    Made of names assigned to values outside of any class or function block.
- The f3 function local scope

    Made of names assigned to values as parameters, or directly in the body of the function.

### Closure 

A closure is a function—let's call it f—with an extended scope that encompasses
variables referenced in the body of f that are not global variables or local variables
of f. Such variables must come from the local scope of an outer function that
encompasses f.

Consider an avg function to compute the mean of an ever-growing series of values;
for example, the average closing price of a commodity over its entire history. Every
day a new price is added, and the average is computed taking into account all prices
so far.

```
>>> avg(10)
10.0
>>> avg(11)
10.5
>>> avg(12)
11.0
```

Where does avg come from, and where does it keep the history of previous values?

One possible implementation is this:

In [50]:
def make_averager():
    series = []
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    return averager

In [51]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


But where does the avg function in the example find the
series?

Note that series is a local variable of make_averager because the assignment series
= [] happens in the body of that function. But when avg(10) is called,
make_averager has already returned, and its local scope is long gone.

Within averager, series is a free variable. This is a technical term meaning a variable
that is not bound in the local scope. See Figure below:

<img width=600 src="https://cs.westminstercollege.edu/~jingsai/courses/CMPT300J/handouts/closure.png">

To summarize: a closure is a function that retains the bindings of the free variables
that exist when the function is defined, so that they can be used later when the function
is invoked and the defining scope is no longer available.

### nonlocal

Our previous implementation of make_averager was not efficient. In previous code,
we stored all the values in the historical series and computed their sum every time
averager was called. A better implementation would only store the total and the
number of items so far, and compute the mean from these two numbers.

In [52]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        count += 1
        total += new_value
        return total / count
    return averager

**Question**

Is there any error in the code if running code below?

In [53]:
avg = make_averager()
avg(10)

UnboundLocalError: ignored

The problem is that the statement count += 1 actually means the same as count =
count + 1, when count is a number or any immutable type. So we are actually
assigning to count in the body of averager, and that makes it a local variable.

To work around this, the nonlocal keyword was introduced in Python 3. It lets you
declare a variable as a free variable even when it is assigned within the function.

In [ ]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

In [ ]:
avg = make_averager()
avg(10)

If we were to use the `global` keyword instead of `nonlocal`, it would indicate that we want to work with the global variable, not the variable from the nearest enclosing scope. If variable were not defined in the global scope, using `global` would result in a `NameError`.

Here's an example illustrating the difference between `nonlocal` and `global`:

In [ ]:
x = 10 # comment out x and restart the runtime -> error 

def outer_function():
    x = 20

    def inner_function():
        nonlocal x
        x += 5
        print("Inner function:", x)

    def inner_function_global():
        global x
        x += 5
        print("Inner function global:", x)

    inner_function()           
    inner_function_global()    

outer_function()
print("Outer function:", x)  

In this example, `outer_function` has a local variable `x` set to 20, and there is a global variable `x` set to 10. `inner_function` uses `nonlocal` to modify the `x` from its enclosing scope, resulting in a value of 25. `inner_function_global` uses `global` to modify the global `x`, resulting in a value of 15. Finally, when we print `x` from the outer scope after calling `outer_function`, it outputs 15, reflecting the modification made by `inner_function_global`.

### Decorator and Closure

In the code below, when `greet` is decorated by `demo_decorator`, `greet` is replaced by `wrapper`. However, the variable `s` is still accessible by `wrapper` even after `demo_decorator` has returned and its local scope has gone.  Thus, `wrapper` is a closure. 

This is why understanding closure is necessary for understanding decorator. 

In [ ]:
def demo_decorator(func):
    s = "Westminster"
    def wrapper():
        return func() + s
    return wrapper

@demo_decorator
def greet():
    return "Hello "

greet()

## References

1. https://www.fluentpython.com/
2. https://docs.python.org/3/library/time.html#time.perf_counter
3. https://stackoverflow.com/questions/25785243/understanding-time-perf-counter-and-time-process-time
4. https://www.programiz.com/python-programming/decorator
5. https://realpython.com/products/python-tricks-book/